In [242]:
"""
Выделение региона и района из данных абитуриентов
"""

'\nВыделение региона и района из данных абитуриентов\n'

In [243]:
import pandas as pd
import openpyxl
import time
from openpyxl.utils.dataframe import dataframe_to_rows
import re



In [244]:
def extract_district(cell):
    """
    Функция для извлечения района или города
    """
    #pattern = re.compile(r'(р-н\s?(?:.+?)),|г\.\s(.+)\s\(')
    pattern = re.compile(r'р-н\s?([а-яА-яёЁ-]+?)[,\s]|(Улан-Удэ)')
    result = re.search(pattern,cell)
    if result:
        if len(result.groups()) == 2:            
            city,district = result.groups()
            if city:
                return city
            else:
                return district
        else:
            return result.groups()
    else:
        return 'Не найдено'
        
    

In [245]:
ab_df = pd.read_excel('data/Абитуриенты.xlsx',sheet_name='Абитуриенты',skiprows=8,usecols=['ФИО','Адрес регистрации','Состояние выбран. конкурса'])

In [246]:
ab_df = ab_df[ab_df['Состояние выбран. конкурса'] == 'Зачислен'] # оставялем только зачисленных

In [247]:
ab_df.shape

(496, 3)

In [248]:
ab_df.drop(columns='Состояние выбран. конкурса',inplace=True)

In [249]:
ab_df.head()

,ФИО,Адрес регистрации
1,Абсаматов Вадим Чимид-Цыренович,"671710, Россия, пгт. Нижнеангарск (03), Респ. ..."
5,Авсеевич Радмила Станиславовна,"671178, Россия, с. Средний Убукун (03), Респ. ..."
9,Агафонов Семен Владимирович,"670050, Россия, г. Улан-Удэ (03), Респ. Буряти..."
10,Агеев Роман Алексеевич,"671404, Россия, п. Майла (03), Респ. Бурятия, ..."
11,Айхлер Павел Владимирович,"671820, Россия, пгт. Наушки (03), Респ. Буряти..."


In [250]:
ab_df['Регион'] = ab_df['Адрес регистрации'].apply(lambda x:re.search(r'\(\d{2}\),(.+?),',x).group(1)) # регион

In [251]:
ab_df.head()

,ФИО,Адрес регистрации,Регион
1,Абсаматов Вадим Чимид-Цыренович,"671710, Россия, пгт. Нижнеангарск (03), Респ. ...",Респ. Бурятия
5,Авсеевич Радмила Станиславовна,"671178, Россия, с. Средний Убукун (03), Респ. ...",Респ. Бурятия
9,Агафонов Семен Владимирович,"670050, Россия, г. Улан-Удэ (03), Респ. Буряти...",Респ. Бурятия
10,Агеев Роман Алексеевич,"671404, Россия, п. Майла (03), Респ. Бурятия, ...",Респ. Бурятия
11,Айхлер Павел Владимирович,"671820, Россия, пгт. Наушки (03), Респ. Буряти...",Респ. Бурятия


In [252]:
ab_df['Район'] = ab_df['Адрес регистрации'].apply(extract_district) # район

In [253]:
ab_df['Район']

1       Северо-Байкальский
5             Селенгинский
9                 Улан-Удэ
10               Хоринский
11              Кяхтинский
               ...        
1231             Хоринский
1232       Красночикойский
1235            Не найдено
1237          Кижингинский
1242              Улан-Удэ
Name: Район, Length: 496, dtype: object

In [254]:
ab_df['Регион'] = ab_df['Регион'].apply(lambda x:x.strip())

In [255]:
reg_df = ab_df['Регион'].value_counts().to_frame()

In [256]:
reg_df

,Регион
Респ. Бурятия,456
край Забайкальский,37
обл. Иркутская,3


In [257]:
ab_df.shape

(496, 4)

In [258]:
distrct_df = ab_df[ab_df['Регион'] == 'Респ. Бурятия']

In [259]:
distrct_df

,ФИО,Адрес регистрации,Регион,Район
1,Абсаматов Вадим Чимид-Цыренович,"671710, Россия, пгт. Нижнеангарск (03), Респ. ...",Респ. Бурятия,Северо-Байкальский
5,Авсеевич Радмила Станиславовна,"671178, Россия, с. Средний Убукун (03), Респ. ...",Респ. Бурятия,Селенгинский
9,Агафонов Семен Владимирович,"670050, Россия, г. Улан-Удэ (03), Респ. Буряти...",Респ. Бурятия,Улан-Удэ
10,Агеев Роман Алексеевич,"671404, Россия, п. Майла (03), Респ. Бурятия, ...",Респ. Бурятия,Хоринский
11,Айхлер Павел Владимирович,"671820, Россия, пгт. Наушки (03), Респ. Буряти...",Респ. Бурятия,Кяхтинский
...,...,...,...,...
1220,Шитин Даниил Максимович,"671404, Россия, п. Майла (03), Респ. Бурятия, ...",Респ. Бурятия,Хоринский
1224,Шиханов Евгений Анатольевич,"670011, Россия, г. Улан-Удэ (03), Респ. Буряти...",Респ. Бурятия,Улан-Удэ
1231,Юдаев Владимир Иванович,"671401, Россия, с. Георгиевское (03), Респ. Бу...",Респ. Бурятия,Хоринский
1237,Ямпилов Саян Зоригович,"671450, Россия, с. Кижинга (03), Респ. Бурятия...",Респ. Бурятия,Кижингинский


In [260]:
distrct_df = distrct_df['Район'].value_counts().to_frame()

In [261]:
with pd.ExcelWriter('data/Распределение по регионам и районам.xlsx') as writer:
    reg_df.to_excel(writer,sheet_name='По регионам')
    distrct_df.to_excel(writer,sheet_name='ПО районам')